# Models

> LangChain LLMS for use by the Alhazen Research Assistant. 

These include the following models:

* Lllama.cpp
* Ollama
* HuggingFace Pipelines
* OpenAI GPT-3.5
* OpenAI GPT-4.0

In [ ]:
#| default_exp models

In [5]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export

from langchain.llms import LlamaCpp, OpenAI

from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.tools import BaseTool
from langchain.utilities import SQLDatabase
import os

In [13]:
#| export

# download GGUF files from HuggingFace URL and save it to disk in defined directory, return local file path
def get_cached_gguf(url, local_dir):
    import requests
    import os
    import shutil
    from tqdm import tqdm
    from pathlib import Path

    # create local directory if not exists
    Path(local_dir).mkdir(parents=True, exist_ok=True)

    # download file
    local_filename = url.split('/')[-1]
    local_filepath = os.path.join(local_dir, local_filename)
    if not os.path.exists(local_filepath):
        r = requests.get(url, stream=True)
        with open(local_filepath, 'wb') as f:
            file_size = int(r.headers['content-length'])
            chunk_size = 1000
            with tqdm(ncols=100, desc="Downloading", total=file_size, unit_scale=True) as pbar:
                for chunk in r.iter_content(chunk_size=chunk_size):
                    f.write(chunk)
                    pbar.update(chunk_size)
    return local_filepath

In [7]:
#| export
def instantiate_llms(langchain_model, **kwargs):

    model = None
    if langchain_model == "LlamaCpp":

        callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
        n_gpu_layers = kwargs.get('n_gpu_layers', 1)
        n_batch = kwargs.get('n_batch', 512)  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
        url = kwargs.get('gguf_url', 'https://huggingface.co/TheBloke/Llama-2-70B-chat-GGUF/resolve/main/llama-2-70b-chat.Q5_K_M.gguf')
        local_dir = os.environ.get('model_dir', None)
        model_path = get_cached_gguf(url, local_dir)
        n_ctx = kwargs.get('n_ctx', 4096)

        model = LlamaCpp(
            model_path=model_path,
            n_ctx=n_ctx,
            n_gpu_layers=n_gpu_layers,
            n_batch=n_batch,
            callback_manager=callback_manager,
            f16_kv=True,
            verbose=True, # Verbose is required to pass to the callback manager
        )        

    elif langchain_model == "GPT3.5":

        model = OpenAI(openai_api_key=os.environ['OPEN_API_KEY'])
 
    else:

        raise ValueError(f"Unknown model {langchain_model}")

    return model


In [10]:
'''
from langchain.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)
'''

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()